# 03 Minimal Web API

**autor: Erik Král ekral@utb.cz**

---

V tomto materiálu si probereme práci s webovými službami s pomocí Minimal Web API.

Web API (Web Application Programming Interface) je sada pravidel a protokolů umožňující komunikovat programům prostřednictvím internetu. REST (Representation State Transfer) je druh Web API a představuj architektonický styl použivající standartní HTTP metody (GET, POST, PUT a DELETE) zpřístpňující endpoity identifikvané pomocí URI. Pro přenos dat využívá přitom především format JSON.

Minimal Web API je zjednodušený způsob tvorby HTTP API pomocí ASP.NET Core.

https://learn.microsoft.com/en-us/aspnet/core/tutorials/min-web-api

https://learn.microsoft.com/en-us/aspnet/core/fundamentals/minimal-apis/responses?view=aspnetcore-9.0#typedresults-vs-results

Nejprve si nainstalujeme nuget pomocí kterého bude moct hostovat Minimal Web API přímo v tomto notebooku.

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AspNetCore,*-*"

Následující kód vrátí na metodu GET "Hello World" spolu s počtem volání této metody. Představuje nejjednodušší program v Minimal Web API.

In [ ]:
#!aspnet

var builder = WebApplication.CreateBuilder();

var app = builder.Build();

int x = 0;
app.MapGet("/", () => $"Hello World {++x}");

await app.StartAsync();


Server zastavíme následujícím příkazem:

In [ ]:
await app.StopAsync();

Nyní si na příkladech probereme jednotlivé metody GET, POST, PUT a DELETE a využijeme při tom Entity Framework.

Použijeme nuget balíček [Microsoft.EntityFrameworkCore.Sqlite](https://www.nuget.org/packages/microsoft.entityframeworkcore.sqlite), který přidá do notebook EF providera pro databází Sqlite.

In [ ]:
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "nuget: Microsoft.Extensions.DependencyInjection"
#r "nuget: Microsoft.AspNetCore.Http"
#r "nuget: Microsoft.AspNetCore.Http.Abstractions"

Nadefinujeme si metodu `GetConnectionString`, která vrátí connection string s umístěním souboru databáze do dokumentů uživatele, tak abychom soubor rychle našli.

In [ ]:
using System.IO;
using Microsoft.Data.Sqlite;

static string GetConnectionString(string fileName)
{
    var folder = Environment.SpecialFolder.MyDocuments;
    string folderPath = Environment.GetFolderPath(folder);
    string filePath = Path.Join(folderPath, fileName);

    SqliteConnectionStringBuilder csb = new SqliteConnectionStringBuilder
    {
        DataSource = filePath
    };

    string connectionString = csb.ConnectionString;

    return connectionString;
}

GetConnectionString("students.db")

Nadefinujeme si třídu `Student`:

In [ ]:
public class Student
{
    public int StudentId {get; set;}
    public required string Jmeno {get; set;}
    public required bool Studuje {get;set;}
}

Dále si nadefinujeme `DbContext`:

In [ ]:
using Microsoft.EntityFrameworkCore;

public class StudentContext : DbContext
{
    public DbSet<Student> Studenti { get; set; }

    public StudentContext(DbContextOptions<StudentContext> options) : base(options)
    {
        
    }
}

Nyní si můžeme nadefinovat endpointy:

In [ ]:
#!aspnet

using Microsoft.EntityFrameworkCore;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.AspNetCore.Http;

var builder = WebApplication.CreateBuilder();
builder.Services.AddDbContext<StudentContext>(op => op.UseSqlite(""));
var app = builder.Build();

int x = 0;
app.MapGet("/", () => $"Hello World {++x}");

await app.StartAsync();

static Task<Microsoft.AspNetCore.Http.IResult> GetAllStudents(StudentContext context)
{
    return Microsoft.AspNetCore.TypedResults.OK(context.Studenti.ToArrayAsync());
}

In [ ]:
await app.StopAsync();